### Intersection of mrcanavar, PacBio CCS, and ONT high coverage regions to identify potential CNV

Generating excessive coverage bed files using calculations from mosdepth for both PacBio CCS 15kb_20kb merged and ONT bam files.

combined_GRCh38.bam is from https://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/giab/data/AshkenazimTrio/HG003_NA24149_father/UCSC_Ultralong_OxfordNanopore_Promethion/HG003_GRCh38_ONT-UL_UCSC_20200508.bam


HG003.SequelII.merged_15kb_20kb.pbmm2.GRCh38.haplotag.10x.bam is from https://drive.google.com/file/d/1mC5n4kGZT6PBpBG40Pml7IHxfGPZV4oh/view?usp=sharing


AJtrio-HG003.hg38.300x.bam.bilkentuniv.072319.dups.bed is from https://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/giab/data/AshkenazimTrio/analysis/BilkentUni_mrCaNaVaR_GRCh38_07242019/AJtrio-HG003.hg38.300x.bam.bilkentuniv.072319.dups.bed.gz


pbsv_HG003_SequelII.GRCh38.vcf.gz is from https://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/giab/data/AshkenazimTrio/analysis/PacBio_CCS_15kb_20kb_chemistry2_10312019/pbsv/pbsv_HG003_SequelII.GRCh38.vcf.gz


convert_mosdepth_to_excessive_coverage.py is at the end of this notebook

### mosdepth commands

In [ ]:
mosdepth -b 1000 -x --mapq 20 --no-per-base HG003.SequelII.merged_15kb_20kb.pbmm2.GRCh38.haplotag.10x_1000_window_size HG003.SequelII.merged_15kb_20kb.pbmm2.GRCh38.haplotag.10x.bam


samtools view -1 -F 0x100 combined_GRCh38.bam -h > filtered_combined_GRCh38.bam

mosdepth -b 1000 -x --no-per-base filtered_combined_GRCh38_1000_window_size filtered_combined_GRCh38.bam

In [ ]:
### Find coverage levels of excessive coverage on CCS and ONT data in R

In [ ]:
chr_1_22 <- c("chr1","chr2","chr3","chr4","chr5","chr6","chr7","chr8","chr9","chr10","chr11","chr12","chr13","chr14","chr15","chr16","chr17","chr18","chr19","chr20","chr21","chr22")
mosdepth_CCS_15kb_20kb_merged_1000_window_size = read.delim("HG003.SequelII.merged_15kb_20kb.pbmm2.GRCh38.haplotag.10x_1000_window_size.regions.bed", col.names = c("CHR","START","END","DEPTH"))
mosdepth_CCS_15kb_20kb_merged_1000_window_size_chr_1_22 <- mosdepth_CCS_15kb_20kb_merged_1000_window_size[which(mosdepth_CCS_15kb_20kb_merged_1000_window_size[,"CHR"] %in% chr_1_22),] 
quantile(mosdepth_CCS_15kb_20kb_merged_1000_window_size_chr_1_22[,"DEPTH"])


IQR(mosdepth_CCS_15kb_20kb_merged_1000_window_size_chr_1_22[,"DEPTH"])
#: 16.92
(quantile(mosdepth_CCS_15kb_20kb_merged_1000_window_size_chr_1_22[,"DEPTH"])[3]/2)*2.5
#: 81.325


mosdepth_ONT_1000_window_size = read.delim("filtered_combined_GRCh38_1000_window_size.regions.bed", col.names = c("CHR","START","END","DEPTH"))
mosdepth_ONT_1000_window_size_chr_1_22 <- mosdepth_ONT_1000_window_size[which(mosdepth_ONT_1000_window_size[,"CHR"] %in% chr_1_22),]

quantile(mosdepth_ONT_1000_window_size_chr_1_22[,"DEPTH"])

IQR(mosdepth_ONT_1000_window_size_chr_1_22[,"DEPTH"])
#: 16.01
(quantile(mosdepth_ONT_1000_window_size_chr_1_22[,"DEPTH"])[3]/2)*2.5
#: 105.9625


In [ ]:
### GRCh38_HG003_mrcanavar_intersect_ccs_1000_window_size_cnv_threshold_intersect_ont_1000_window_size_cnv_threshold.bed


### What this does: find potential CNVs in HG3 from intersecting coverage files from PacBio HiFi, ONT, and Illumina data. This generates PacBio HiFi excessive coverage bed, intersect with mrCaNaVar dups bed, generates ONT excessive coverage bed, intersects to these all to generate exclusion bed

In [ ]:
python convert_mosdepth_to_excessive_coverage.py --input HG003.SequelII.merged_15kb_20kb.pbmm2.GRCh38.haplotag.10x_1000_window_size.regions.bed --output HG003.SequelII.merged_15kb_20kb.pbmm2.GRCh38.haplotag.10x_1000_window_size_excessive_coverage_cnv_threshold.bed --threshold 81.325

bedtools intersect -a AJtrio-HG003.hg38.300x.bam.bilkentuniv.072319.dups.bed -b HG003.SequelII.merged_15kb_20kb.pbmm2.GRCh38.haplotag.10x_1000_window_size_excessive_coverage_cnv_threshold.bed > mrcanavar_intersect_ccs_cnv_threshold.bed

python convert_mosdepth_to_excessive_coverage.py --input filtered_combined_GRCh38_1000_window_size.regions.bed --output filtered_combined_GRCh38_1000_window_size_excessive_coverage_cnv_threshold.bed --threshold 105.9625

bedtools intersect -a mrcanavar_intersect_ccs_cnv_threshold.bed -b filtered_combined_GRCh38_1000_window_size_excessive_coverage_cnv_threshold.bed > mrcanavar_intersect_ccs_window_size_cnv_threshold_intersect_ont_window_size_cnv_threshold.bed

cp mrcanavar_intersect_ccs_window_size_cnv_threshold_intersect_ont_window_size_cnv_threshold.bed GRCh38_HG003_mrcanavar_intersect_ccs_1000_window_size_cnv_threshold_intersect_ont_1000_window_size_cnv_threshold.bed


In [ ]:
### union_HG003_CCS_15kb_20kb_merged_ONT_1000_window_size_combined_elliptical_outlier_threshold.bed

### What this does: Find another set of potential CNVs from computing a coverage threshold using an elliciptal outlier for PacBio HiFi and ONT CNV file steps in R

mosdepth_CCS_15kb_20kb_merged_1000_window_size = read.delim("HG003.SequelII.merged_15kb_20kb.pbmm2.GRCh38.haplotag.10x_1000_window_size.regions.bed", col.names = c("CHR","START","END","DEPTH"))

mosdepth_ONT_1000_window_size = read.delim("filtered_combined_GRCh38_1000_window_size.regions.bed", col.names = c("CHR","START","END","DEPTH"))

chr_1_22 <- c("chr1","chr2","chr3","chr4","chr5","chr6","chr7","chr8","chr9","chr10","chr11","chr12","chr13","chr14","chr15","chr16","chr17","chr18","chr19","chr20","chr21","chr22")

mosdepth_CCS_15kb_20kb_merged_1000_window_size_chr_1_22 <- mosdepth_CCS_15kb_20kb_merged_1000_window_size[which(mosdepth_CCS_15kb_20kb_merged_1000_window_size[,"CHR"] %in% chr_1_22),]

mosdepth_ONT_1000_window_size_chr_1_22 <- mosdepth_ONT_1000_window_size[which(mosdepth_ONT_1000_window_size[,"CHR"] %in% chr_1_22),]

df_mosdepth_CCS_15kb_20kb_merged_1000_window_size <- data.frame(mosdepth_CCS_15kb_20kb_merged_1000_window_size_chr_1_22) 

df_mosdepth_ONT_1000_window_size <- data.frame(mosdepth_ONT_1000_window_size_chr_1_22)

df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined <- df_mosdepth_CCS_15kb_20kb_merged_1000_window_size

df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined[,5] <- mosdepth_ONT_1000_window_size_chr_1_22[,4]

colnames(df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined) <- c("CHR", "START", "END", "CCS_DEPTH", "ONT_DEPTH")

df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined_elliptical_values <- df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined 

medianccsdepth = median(mosdepth_CCS_15kb_20kb_merged_1000_window_size_chr_1_22[,"DEPTH"]) 

medianontdepth = median(mosdepth_ONT_1000_window_size_chr_1_22[,"DEPTH"]) 

df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined_elliptical_values[,6] <- sqrt(((df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined[,4]/medianccsdepth)^2 + (df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined[,5]/medianontdepth)^2)/2) 

threshold_ellipctial_outlier = unname(quantile(df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined_elliptical_values[,6])[4]+(1.5*IQR(df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined_elliptical_values[,6])[1]))

df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined_outliers <- df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined_elliptical_values[which(df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined_elliptical_values[,6] > threshold_ellipctial_outlier),]

write.csv(df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined_outliers, file = "GRCh38_union_HG003_CCS_15kb_20kb_merged_ONT_1000_window_size_combined_elliptical_outlier_threshold.bed", row.names = FALSE)


### pbsv_HG003_SequelII.GRCh38_gt49bp_slop50_repeatexpanded_slop25percent.bed

### What this does: find SV calls from PacBio HiFi that will be excluded from HG3 in addition to the v0.6 GIAB SV calls

In [ ]:
#Form bed to exclude SV regions from HG003 v4.2.1 benchmark

zgrep -v ^# pbsv_HG003_SequelII.GRCh38.vcf.gz | awk '{FS=OFS="\t"} {  if(length($4)>49 || length($5)>49) print $1,$2-50,$2+length($4)+50} '  > pbsv_HG003_SequelII.GRCh38_gt49bp_slop50.bed

intersectBed -wa -a GRCh38_AllTandemRepeatsandHomopolymers_slop5.bed.gz -b pbsv_HG003_SequelII.GRCh38_gt49bp_slop50.bed | multiIntersectBed -i stdin pbsv_HG003_SequelII.GRCh38_gt49bp_slop50.bed | mergeBed -i stdin -d 1000 | awk '{FS=OFS="\t"} { slop=int(0.25*($3-$2)); if(slop>5000) slop=5000; print $1, $2-slop, $3+slop}' | awk '{FS=OFS="\t"} { if($2<0) $2=0; print}' > pbsv_HG003_SequelII.GRCh38_gt49bp_slop50_repeatexpanded_slop25percent.bed

#### convert_mosdepth_to_excessive_coverage.py script

import argparse

parser = argparse.ArgumentParser(description="Subset bed file to callable regions only")
parser.add_argument('--input_file', metavar="I", type=str, nargs="+", help="input bed file")
parser.add_argument('--output_file', metavar="O", type=str, nargs="+", help="output file")
parser.add_argument('--threshold', metavar="T", type=str, nargs="+", help="input threshold")
args = parser.parse_args()

f = open(args.input_file[0], "r") 
f_lines = f.readlines()

f_out = open(args.output_file[0], "w+")
threshold = float(args.threshold[0])

for line in f_lines:   
    if "DEPTH" in line: 
        continue
    line_split = line.split("\t")
    depth_field = float(line_split[3])
    if depth_field > threshold:
        f_out.write(line)
        f_out.flush()  

f.close()
f_out.close()